In [21]:
import re
import json
import param
import pickle
import logging
import operator
import unicodedata
import stctokenizer
import numpy as np
from pprint import pprint
from random import shuffle
from collections import Counter
from collections import defaultdict
from gensim.models.keyedvectors import KeyedVectors

from gensim.models import word2vec
from gensim import models

from nltk.corpus import stopwords
sw = stopwords.words("english")

doclen = param.doclen
embsize = param.embsize
max_sent = param.max_sent
NDclasses = param.NDclasses
stctokenizer = stctokenizer.STCTokenizer()

In [25]:
def turn2mask(turns):
    # {'CNUG*': 0, 'CNUG': 1, 'CNaN': 2, 'CNUG0': 3, 'HNUG*': 4, 'HNUG': 5, 'HNaN': 6}
    all_dialog_masks = []
    for turn in turns:
        dialog_mask = []
        for i in range(max_sent):
            if i < turn:
                if i % 2 == 0:  # customer
                    dialog_mask.append(np.concatenate((np.ones(4), np.zeros(3))))
                else:  # helpdesk
                    dialog_mask.append(np.concatenate((np.zeros(4), np.ones(3))))
            else:
                dialog_mask.append(np.zeros(max_sent))

        dialog_mask = np.asarray(dialog_mask)
        all_dialog_masks.append(np.asarray(dialog_mask.copy()))
    return all_dialog_masks

In [26]:
def user_input(texts):
    if type(texts) != list:
        assert False, 'Input should be a list'
    print('Importing BertClient')
    from bert_serving.client import BertClient
    bc = BertClient(ip='140.115.54.42')
    
    X = []
    turns = []
    bertX = []
    
    remove_stopwords = False
    to_lower = True
    token_type = 'nltk'
    emb = 'stc' # glove or stc

    maxlen = 0
    dialogbertX = []

    for text in texts:
        text = '.' if text == '' else text
        tokens = stctokenizer.tokenize(token_type, text, remove_stopwords, to_lower)
        vec = np.reshape(bc.encode([text]), 1024)
        dialogbertX.append(vec)

    turns.append(len(dialogbertX))

    # Pending with zero for dialogs with turns < 7
    while len(dialogbertX) < max_sent:
        dialogbertX.append(np.zeros([1024, ]))

    bertX.append(np.asarray(dialogbertX))
    masks = turn2mask(turns)
    return np.asarray(bertX), turns, masks

In [27]:
inputX, input_turns, input_masks = user_input(['Hi how are you doing', 'Im good thank you'])

Importing BertClient


In [29]:
inputX.shape

(1, 7, 1024)